In [1]:
from sqlalchemy import create_engine 
from dotenv import load_dotenv
import os
import pandas as pd
import psycopg2

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
if DATABASE_URL and DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)
    
engine = create_engine(DATABASE_URL, echo = False)


In [6]:
df = pd.read_csv("output.csv", index_col=0)

df.to_sql('data', con=engine, if_exists="append")


42

In [7]:
query = f"""SELECT * FROM data"""

In [8]:
import psycopg2

con = psycopg2.connect(DATABASE_URL)
cur = con.cursor()

In [39]:
test_data = pd.read_csv("output.csv")

In [48]:
test_data.head(1)

,category,title,link,organiser,date,day,time,address
0,VISUAL ARTS,BreatheWater,https://www.esplanade.com/festivals-and-series...,Parvathi Nayar (India),25 Mar – 24 Jul 2022,Fri – Sun,All Day,Theatre Street Cones


In [49]:
df.head(1)

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [50]:
data_new = pd.concat([test_data, df], axis=0)

In [52]:
data_new.tail()

,category,title,link,organiser,date,day,time,address
38,THEATRE,Recipes for Wayfinding (Care Index) Installation,https://www.esplanade.com/festivals-and-series...,Alecia Neo (Singapore),28 Jul – 24 Sep 2022,Thu – Sat,All Day,Esplanade Roof Terrace
39,THEATRE,In Their Own Words: Writing Verbatim Plays,https://www.esplanade.com/festivals-and-series...,Haresh Sharma (Singapore),27-Aug-22,Sat,12pm,library@esplanade
40,THEATRE,Performing Grief,https://www.esplanade.com/festivals-and-series...,"Mohd Fared Jainal, Chong Tze Chien, Jolene Chi...",10-Sep-22,Sat,3pm,library@esplanade
41,THEATRE,In Conversation: And So The Conversations Cont...,https://www.esplanade.com/festivals-and-series...,Checkpoint Theatre (Singapore),24-Sep-22,Sat,3pm,library@esplanade
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [44]:
test = [] 
test.append({'category': "category_test",
                    'title': 'title_text',
                    'link': 'link_text',
                    'organiser': 'organiser_text',
                    'date': 'date_text',
                    'day': 'day_text',
                    'time': 'time_text',
                    'address': 'address_text'})

In [45]:
df = pd.DataFrame(test)

In [46]:
df

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [21]:
def check_update(df, new_titles, con):
    query = f"""SELECT * FROM data"""
    old_title = list(pd.read_sql(query, con).loc[:,"title"])
    print("Number of old titles: {}".format(len(old_title)))
    for title in list(df["title"]):
        if title not in old_title:
            new_titles.append(title)
    print("Number of new new titles: {}".format(len(new_titles)))
    return new_titles

In [54]:
new_titles = []
update = check_update(data_new, new_titles, con)

c:\Users\joann\Anaconda3\envs\tensorflow-gpu\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Number of old titles: 42
Number of new new titles: 1


In [55]:
df_update = df[df["title"].isin(update)]

In [56]:
df_update

,category,title,link,organiser,date,day,time,address
0,category_test,title_text,link_text,organiser_text,date_text,day_text,time_text,address_text


In [ ]:
df_update.to_sql('data', con=engine, if_exists="append", index=False)

In [1]:
import pandas as pd
import time
import csv
import os
import sys

# Selenium Web Scraping
from selenium import webdriver
from selenium.webdriver.chrome import options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.service import Service
from dotenv import load_dotenv
from tqdm import tqdm
import telegram

# SQL Database
import psycopg2
from sqlalchemy import create_engine 

In [2]:
load_dotenv()

True

In [5]:
url = "https://www.esplanade.com/whats-on?performanceNature=Free+Programme"
API_KEY = os.getenv("API_KEY")
CHAT_ID = os.getenv("CHAT_ID")
DATABASE_URL = os.getenv("DATABASE_URL")

In [49]:
 # setting up options
chrome_options = webdriver.ChromeOptions()
#chrome_options.binary_location = os.getenv("GOOGLE_CHROME_BIN")
#chrome_options.add_argument("--headless")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome("chromedriver.exe", chrome_options=chrome_options)
#driver = webdriver.Chrome(service=Service(os.getenv("CHROMEDRIVER_PATH")), options=chrome_options)

C:\Users\joann\AppData\Local\Temp\ipykernel_12196\1715630161.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=chrome_options)
C:\Users\joann\AppData\Local\Temp\ipykernel_12196\1715630161.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=chrome_options)


In [50]:
driver.get(url)

In [51]:
current_site = driver.current_url

In [61]:
load_more = driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']")[0].text
while load_more == "Load more":
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']")[0].click()
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    load_more = driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']")[0].text


trying
Waiting
trying
Waiting
trying
Waiting


In [54]:
driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']")[0].text

'Load more'

In [40]:
driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']")[0].click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0114D953+2414931]
	Ordinal0 [0x010DF5E1+1963489]
	Ordinal0 [0x00FCC580+836992]
	Ordinal0 [0x00FFA4F3+1025267]
	Ordinal0 [0x00FF0293+983699]
	Ordinal0 [0x0101449C+1131676]
	Ordinal0 [0x00FEFC74+982132]
	Ordinal0 [0x010146B4+1132212]
	Ordinal0 [0x01024812+1198098]
	Ordinal0 [0x010142B6+1131190]
	Ordinal0 [0x00FEE860+976992]
	Ordinal0 [0x00FEF756+980822]
	GetHandleVerifier [0x013BCC62+2510274]
	GetHandleVerifier [0x013AF760+2455744]
	GetHandleVerifier [0x011DEABA+551962]
	GetHandleVerifier [0x011DD916+547446]
	Ordinal0 [0x010E5F3B+1990459]
	Ordinal0 [0x010EA898+2009240]
	Ordinal0 [0x010EA985+2009477]
	Ordinal0 [0x010F3AD1+2046673]
	BaseThreadInitThunk [0x75D16739+25]
	RtlGetFullPathName_UEx [0x770D8FEF+1215]
	RtlGetFullPathName_UEx [0x770D8FBD+1165]


In [48]:
driver.find_elements(By.XPATH, "//div[@x-show='showLoadMore']/button/span[@x-show='!inProgress']")[0].text

''

In [39]:
driver.find_elements(By.XPATH, "//button[@class='flex justify-center items-center text-xl font-bold bg-black text-white h-max text-center font-bold cursor-pointer inline-block transition ease-in-out duration-200 py-3 px-6 min-w-[7.86rem] md:min-w-[11.46rem] max-w-[24.57rem] md:max-w-[22.78rem] rounded hover:bg-gray-2']")[0].click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0114D953+2414931]
	Ordinal0 [0x010DF5E1+1963489]
	Ordinal0 [0x00FCC580+836992]
	Ordinal0 [0x00FFA4F3+1025267]
	Ordinal0 [0x00FF0293+983699]
	Ordinal0 [0x0101449C+1131676]
	Ordinal0 [0x00FEFC74+982132]
	Ordinal0 [0x010146B4+1132212]
	Ordinal0 [0x01024812+1198098]
	Ordinal0 [0x010142B6+1131190]
	Ordinal0 [0x00FEE860+976992]
	Ordinal0 [0x00FEF756+980822]
	GetHandleVerifier [0x013BCC62+2510274]
	GetHandleVerifier [0x013AF760+2455744]
	GetHandleVerifier [0x011DEABA+551962]
	GetHandleVerifier [0x011DD916+547446]
	Ordinal0 [0x010E5F3B+1990459]
	Ordinal0 [0x010EA898+2009240]
	Ordinal0 [0x010EA985+2009477]
	Ordinal0 [0x010F3AD1+2046673]
	BaseThreadInitThunk [0x75D16739+25]
	RtlGetFullPathName_UEx [0x770D8FEF+1215]
	RtlGetFullPathName_UEx [0x770D8FBD+1165]


In [26]:
driver.find_elements(By.CLASS_NAME, "w-full text-center mb-3 md:mb-0 mt-7 md:mt-10 flex justify-center")

InvalidSelectorException: Message: invalid selector: An invalid or illegal selector was specified
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0114D953+2414931]
	Ordinal0 [0x010DF5E1+1963489]
	Ordinal0 [0x00FCC6B8+837304]
	Ordinal0 [0x00FCF0B4+848052]
	Ordinal0 [0x00FCEF72+847730]
	Ordinal0 [0x00FCF200+848384]
	Ordinal0 [0x00FF9215+1020437]
	Ordinal0 [0x00FF979B+1021851]
	Ordinal0 [0x0102653C+1205564]
	Ordinal0 [0x010144E4+1131748]
	Ordinal0 [0x01024812+1198098]
	Ordinal0 [0x010142B6+1131190]
	Ordinal0 [0x00FEE860+976992]
	Ordinal0 [0x00FEF756+980822]
	GetHandleVerifier [0x013BCC62+2510274]
	GetHandleVerifier [0x013AF760+2455744]
	GetHandleVerifier [0x011DEABA+551962]
	GetHandleVerifier [0x011DD916+547446]
	Ordinal0 [0x010E5F3B+1990459]
	Ordinal0 [0x010EA898+2009240]
	Ordinal0 [0x010EA985+2009477]
	Ordinal0 [0x010F3AD1+2046673]
	BaseThreadInitThunk [0x75D16739+25]
	RtlGetFullPathName_UEx [0x770D8FEF+1215]
	RtlGetFullPathName_UEx [0x770D8FBD+1165]


In [9]:
pages_text = driver.find_element(By.XPATH, "//div[@class='paging-container']").text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='paging-container']"}
  (Session info: chrome=102.0.5005.115)
Stacktrace:
Backtrace:
	Ordinal0 [0x0114D953+2414931]
	Ordinal0 [0x010DF5E1+1963489]
	Ordinal0 [0x00FCC6B8+837304]
	Ordinal0 [0x00FF9500+1021184]
	Ordinal0 [0x00FF979B+1021851]
	Ordinal0 [0x01026502+1205506]
	Ordinal0 [0x010144E4+1131748]
	Ordinal0 [0x01024812+1198098]
	Ordinal0 [0x010142B6+1131190]
	Ordinal0 [0x00FEE860+976992]
	Ordinal0 [0x00FEF756+980822]
	GetHandleVerifier [0x013BCC62+2510274]
	GetHandleVerifier [0x013AF760+2455744]
	GetHandleVerifier [0x011DEABA+551962]
	GetHandleVerifier [0x011DD916+547446]
	Ordinal0 [0x010E5F3B+1990459]
	Ordinal0 [0x010EA898+2009240]
	Ordinal0 [0x010EA985+2009477]
	Ordinal0 [0x010F3AD1+2046673]
	BaseThreadInitThunk [0x75D16739+25]
	RtlGetFullPathName_UEx [0x770D8FEF+1215]
	RtlGetFullPathName_UEx [0x770D8FBD+1165]
